In [1]:
import pandas as pd
import numpy as np
from linearmodels import PanelOLS,RandomEffects
from scipy import stats
import statsmodels.api as sm

In [2]:
def x_day_return(series,x):
    
    return series.pct_change(periods=x)

In [3]:
data_directory = "data/"
file = "ab_stock_cross_section.csv"

In [4]:
df = pd.read_csv(data_directory+file)
df = df.iloc[:,1:]
df["date"] = pd.to_datetime(df["date"])
df.index = df["date"]

In [6]:
x = df.iloc[0,:]

In [11]:
df = df[df["date"]>="2002-01-01"]

In [12]:
def ret(x):
    
    series = x["a_price"]
    ret = x_day_return(series,10).shift(-10)
    return ret

In [13]:
x_day_return_series = df.groupby("unique_id").apply(ret)

In [14]:
index_df = x_day_return_series.index.to_frame()

index_df["a_10_day_return"]= x_day_return_series


In [15]:
index_df.index = range(0,index_df.shape[0])
df.index = range(0,df.shape[0])

In [16]:
merge_df = df.merge(index_df,on=["date","unique_id"])

In [17]:
def ret(x):
    
    series = x["a_price"]
    ret = x_day_return(series,5).shift(-5)
    return ret

In [18]:
df.index = df["date"]

In [19]:
x_day_return_series = df.groupby("unique_id").apply(ret)

In [20]:
index_df = x_day_return_series.index.to_frame()

index_df["a_5_day_return"]= x_day_return_series


In [21]:
index_df.index = range(0,index_df.shape[0])

In [22]:
merge_df = merge_df.merge(index_df,on=["date","unique_id"])

In [23]:
def ret(x):
    
    series = x["a_price"]
    ret = x_day_return(series,20).shift(-20)
    return ret

In [24]:
df.index = df["date"]

In [25]:
x_day_return_series = df.groupby("unique_id").apply(ret)

In [26]:
index_df = x_day_return_series.index.to_frame()

index_df["a_20_day_return"]= x_day_return_series

In [27]:
index_df.index = range(0,index_df.shape[0])

In [28]:
merge_df = merge_df.merge(index_df,on=["date","unique_id"])

In [29]:
merge_df

,unique_id,date,a_price,b_price,ratio,a_ret,b_ret,a_10_day_return,a_5_day_return,a_20_day_return
0,id_51,2002-01-04,12.99,8.870,1.464487,-0.026966,0.000000,-0.131640,-0.052348,-0.038491
1,id_51,2002-01-07,12.83,8.870,1.446449,-0.012317,0.000000,-0.146532,-0.062354,-0.020265
2,id_51,2002-01-08,12.70,8.530,1.488863,-0.010133,-0.038331,-0.123622,-0.081890,-0.014961
3,id_51,2002-01-09,12.57,8.310,1.512635,-0.010236,-0.025791,-0.044551,-0.064439,-0.017502
4,id_51,2002-01-10,12.64,8.500,1.487059,0.005569,0.022864,-0.049842,-0.108386,-0.009494
...,...,...,...,...,...,...,...,...,...,...
348401,id_40,2019-12-25,7.73,0.398,19.422111,-0.001292,0.000000,NaN,NaN,NaN
348402,id_40,2019-12-26,7.68,0.400,19.200000,-0.006468,0.005025,NaN,NaN,NaN
348403,id_40,2019-12-27,7.59,0.400,18.975000,-0.011719,0.000000,NaN,NaN,NaN
348404,id_40,2019-12-30,7.74,0.397,19.496222,0.019763,-0.007500,NaN,NaN,NaN


In [30]:
merge_df["unique_id"] = pd.Categorical(merge_df.unique_id)
merge_df = merge_df.set_index(['unique_id',"date"])

In [31]:
merge_df

a_price  b_price      ratio     a_ret     b_ret  \
unique_id date                                                          
id_51     2002-01-04    12.99    8.870   1.464487 -0.026966  0.000000   
          2002-01-07    12.83    8.870   1.446449 -0.012317  0.000000   
          2002-01-08    12.70    8.530   1.488863 -0.010133 -0.038331   
          2002-01-09    12.57    8.310   1.512635 -0.010236 -0.025791   
          2002-01-10    12.64    8.500   1.487059  0.005569  0.022864   
...                       ...      ...        ...       ...       ...   
id_40     2019-12-25     7.73    0.398  19.422111 -0.001292  0.000000   
          2019-12-26     7.68    0.400  19.200000 -0.006468  0.005025   
          2019-12-27     7.59    0.400  18.975000 -0.011719  0.000000   
          2019-12-30     7.74    0.397  19.496222  0.019763 -0.007500   
          2019-12-31     8.04    0.401  20.049875  0.038760  0.010076   

                      a_10_day_return  a_5_day_return  a_20_day_return  
unique_id date                                                          
id_51     2002-01-04        -0.131640       -0.052348        -0.038491  
          2002-01-07        -0.146532       -0.062354        -0.020265  
          2002-01-08        -0.123622       -0.081890        -0.014961  
          2002-01-09        -0.044551       -0.064439        -0.017502  
          2002-01-10        -0.049842       -0.108386        -0.009494  
...                               ...             ...              ...  
id_40     2019-12-25              NaN             NaN              NaN  
          2019-12-26              NaN             NaN              NaN  
          2019-12-27              NaN             NaN              NaN  
          2019-12-30              NaN             NaN              NaN  
          2019-12-31              NaN             NaN              NaN  

[348406 rows x 8 columns]

In [32]:
merge_df = merge_df.dropna()

In [33]:
exog_vars = ["ratio"]
depe_var = ["a_20_day_return"]
exog = sm.add_constant(merge_df[exog_vars])

In [34]:
exog

const      ratio
unique_id date                        
id_51     2002-01-04    1.0   1.464487
          2002-01-07    1.0   1.446449
          2002-01-08    1.0   1.488863
          2002-01-09    1.0   1.512635
          2002-01-10    1.0   1.487059
...                     ...        ...
id_40     2019-11-27    1.0  19.408740
          2019-11-28    1.0  20.000000
          2019-11-29    1.0  20.697051
          2019-12-02    1.0  20.700809
          2019-12-03    1.0  20.781671

[346632 rows x 2 columns]

In [35]:
mod = PanelOLS(merge_df[depe_var], exog,entity_effects=False,time_effects=False)

In [36]:
re_res = mod.fit(cov_type='robust', cluster_entity=True)

In [37]:
result = re_res.summary

In [38]:
result

Dep. Variable:,a_20_day_return,R-squared:,0.0006
Estimator:,PanelOLS,R-squared (Between):,-0.3655
No. Observations:,346632,R-squared (Within):,0.0011
Date:,"Sun, Dec 20 2020",R-squared (Overall):,0.0006
Time:,19:41:38,Log-likelihood,1.504e+05
Cov. Estimator:,Robust,,
,,F-statistic:,202.52
Entities:,89,P-value,0.0000
Avg Obs:,3894.7,Distribution:,"F(1,346630)"
Min Obs:,0.0000,,
Max Obs:,4296.0,F-statistic (robust):,196.71
